<h1>Liveability</h1>

In [2]:
# Importing necessary libraries
import folium
import geopandas as gpd 
import pandas as pd
import sys

sys.path.insert(0, "../scripts/")
from helper_functions import convert_census_to_postcode

### Read in relevant data

In [3]:
all_property_df = pd.read_csv("../data/curated/categorised_distances.csv")

property_df = all_property_df[["Cost", "Bed", "Bath", "Parking", "Postcode"]]
distances_df = all_property_df[["Bed", "Bath", "Parking", "Postcode", "Railway_Duration", "CBD_Duration", "Post_Office_Duration", "Park_Distance", 
                                "Nearby_Schools"]]

schools_df = pd.read_csv("../data/curated/schools/school_counts.csv")

In [4]:
# View property data
property_df.head()

,Cost,Bed,Bath,Parking,Postcode
0,440.0,1,1,0,3000
1,620.0,1,1,0,3000
2,300.0,1,1,0,3000
3,400.0,1,1,0,3000
4,625.0,2,2,1,3000


In [5]:
# View categorised distances
distances_df.head()

,Bed,Bath,Parking,Postcode,Railway_Duration,CBD_Duration,Post_Office_Duration,Park_Distance,Nearby_Schools
0,1,1,0,3000,1,80.0,1,1,612
1,1,1,0,3000,2,113.5,1,1,630
2,1,1,0,3000,1,70.4,1,1,612
3,1,1,0,3000,2,93.4,1,1,625
4,2,2,1,3000,1,136.8,1,1,608


In [6]:
# View school information
schools_df.head()

,Unnamed: 0,Postcode,Education_Sector,School_Type,Number_of_Schools
0,0,3000,Independent,Secondary,3
1,1,3000,Independent,Special,1
2,2,3003,Catholic,Secondary,1
3,3,3004,Government,Secondary,1
4,4,3004,Government,Special,1


### Calculating the average number of nearby amenities per postcode

In [7]:
avg_cbd_dur = distances_df.groupby(by = "Postcode")["CBD_Duration"].mean()
avg_cbd_dur.head()

Postcode
3000    130.228671
3002    302.665909
3003    228.045312
3004    429.717391
3006    311.195918
Name: CBD_Duration, dtype: float64

In [8]:
avg_station_dur = distances_df.groupby(by = "Postcode")["Railway_Duration"].mean()
avg_station_dur.head()

Postcode
3000    1.482517
3002    1.818182
3003    1.718750
3004    2.565217
3006    2.663265
Name: Railway_Duration, dtype: float64

In [9]:
avg_park_dist = distances_df.groupby(by = "Postcode")["Park_Distance"].mean()
avg_park_dist.head()

Postcode
3000    1.000000
3002    1.000000
3003    1.000000
3004    1.130435
3006    1.066327
Name: Park_Distance, dtype: float64

In [10]:
avg_post_dur = distances_df.groupby(by = "Postcode")["Post_Office_Duration"].mean()
avg_post_dur.head()

Postcode
3000    1.377622
3002    1.909091
3003    2.000000
3004    1.673913
3006    2.306122
Name: Post_Office_Duration, dtype: float64

In [11]:
avg_num_schools = distances_df.groupby(by = "Postcode")["Nearby_Schools"].mean()
avg_num_schools.head()

Postcode
3000    613.013986
3002    628.068182
3003    591.265625
3004    564.804348
3006    630.561224
Name: Nearby_Schools, dtype: float64

In [12]:
# Creating dataframe of average amenities per postcode
amenities_df = pd.DataFrame()
amenities_df["Postcode"] = property_df["Postcode"].unique()
amenities_df["Average Duration to CBD"] = avg_cbd_dur.to_list()
amenities_df["Average Duration to Station"] = avg_station_dur.to_list()
amenities_df["Average Distance to Park"] = avg_park_dist.to_list()
amenities_df["Average Duration to Post Office"] = avg_post_dur.to_list()
amenities_df["Average # of Schools"] = avg_num_schools.to_list()
amenities_df.head()

,Postcode,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools
0,3000,130.228671,1.482517,1.000000,1.377622,613.013986
1,3002,302.665909,1.818182,1.000000,1.909091,628.068182
2,3003,228.045312,1.718750,1.000000,2.000000,591.265625
3,3004,429.717391,2.565217,1.130435,1.673913,564.804348
4,3006,311.195918,2.663265,1.066327,2.306122,630.561224


### Calculating the average number of facilities

In [13]:
avg_bed = property_df.groupby(by = "Postcode")["Bed"].mean()
avg_bed.head()

Postcode
3000    1.601399
3002    1.909091
3003    1.796875
3004    1.771739
3006    1.785714
Name: Bed, dtype: float64

In [14]:
avg_bath = property_df.groupby(by = "Postcode")["Bath"].mean()
avg_bath.head()

Postcode
3000    1.300699
3002    1.318182
3003    1.343750
3004    1.423913
3006    1.469388
Name: Bath, dtype: float64

In [15]:
avg_parking = property_df.groupby(by = "Postcode")["Parking"].mean()
avg_parking.head()

Postcode
3000    0.356643
3002    1.000000
3003    0.609375
3004    1.097826
3006    0.647959
Name: Parking, dtype: float64

In [16]:
# Creating dataframe of average facilities per postcode
facilities_df = pd.DataFrame()
facilities_df["Postcode"] = property_df["Postcode"].unique()
facilities_df["Average # Beds"] = avg_bed.to_list()
facilities_df["Average # Baths"] = avg_bath.to_list()
facilities_df["Average # Parking"] = avg_parking.to_list()
facilities_df.head()

,Postcode,Average # Beds,Average # Baths,Average # Parking
0,3000,1.601399,1.300699,0.356643
1,3002,1.909091,1.318182,1.000000
2,3003,1.796875,1.343750,0.609375
3,3004,1.771739,1.423913,1.097826
4,3006,1.785714,1.469388,0.647959


In [17]:
# Joining facilities and amenities dataframes
amenities_df.set_index("Postcode", inplace = True)
facilities_df.set_index("Postcode", inplace = True)
postcode_property_df = amenities_df.join(facilities_df, on = "Postcode")
postcode_property_df.head()

,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,,,,,
3000,130.228671,1.482517,1.000000,1.377622,613.013986,1.601399,1.300699,0.356643
3002,302.665909,1.818182,1.000000,1.909091,628.068182,1.909091,1.318182,1.000000
3003,228.045312,1.718750,1.000000,2.000000,591.265625,1.796875,1.343750,0.609375
3004,429.717391,2.565217,1.130435,1.673913,564.804348,1.771739,1.423913,1.097826
3006,311.195918,2.663265,1.066327,2.306122,630.561224,1.785714,1.469388,0.647959


### Obtaining census/population data

In [18]:
# Reading in csv files for obtaining census data
sa2_postcode_map = pd.read_csv("../data/curated/sa2_postcode_mapping_2021.csv")
sa2_postcode_map.set_index("sa2_2021", inplace = True)
census_df = pd.read_csv("../data/curated/census_data.csv")

In [19]:
# Using helper function to convert sa2 mapping to postcodes
census_by_postcode_df = convert_census_to_postcode(census_df, sa2_postcode_map, "mean_no_zero")
census_by_postcode_df.head()

,postcode_2021,tot_population_11,tot_population_16,tot_population_21,avg_med_mortg_rep_11,avg_med_mortg_rep_16,avg_med_mortg_rep_21,avg_med_person_inc_11,avg_med_person_inc_16,avg_med_person_inc_21,avg_med_rent_16,avg_med_rent_11,avg_med_rent_21,avg_med_hh_inc_16,avg_med_hh_inc_11,avg_med_hh_inc_21,tot_avg_hh_size_16,tot_avg_hh_size_11,tot_avg_hh_size_21
0,3000,124551,167166,178424,2213.38,2040.38,2040.19,862.18,5483.82,6467.76,395.76,447.06,418.19,1482.53,1896.76,2159.41,1.88,1.97,1.86
1,3002,68729,82804,89023,2357.78,2173.67,2155.22,1091.80,8969.60,10432.90,398.00,460.33,449.67,1709.40,2415.00,2598.80,1.82,1.91,1.87
2,3003,15496,20633,23083,2200.00,2050.00,2085.00,701.50,716.00,1000.00,395.00,418.50,385.50,1466.00,1493.50,1751.00,2.15,2.15,1.95
3,3004,100879,123254,129273,2331.58,2155.67,2149.75,1066.08,7152.46,8339.46,391.15,446.83,440.75,1688.85,2270.46,2471.46,1.83,1.89,1.84
4,3006,21150,30239,36164,2477.25,2217.75,2079.00,1132.40,16783.00,19507.00,406.80,501.00,461.00,1637.20,2883.20,3088.80,1.80,1.92,1.92


In [20]:
# Removing unnecessary columns from census data (only need columns containing data from 2021)
census_by_postcode_df = census_by_postcode_df[["postcode_2021", "tot_population_21", "avg_med_mortg_rep_21", "avg_med_person_inc_21", "avg_med_rent_21", 
                            "avg_med_hh_inc_21", "tot_avg_hh_size_21"]]

# Selecting columns required for assessing liveability
population_df = census_by_postcode_df[["postcode_2021", "tot_avg_hh_size_21"]]
population_df.head()

,postcode_2021,tot_avg_hh_size_21
0,3000,1.86
1,3002,1.87
2,3003,1.95
3,3004,1.84
4,3006,1.92


In [21]:
# Renaming postcode column of census dataframe and setting to index
population_df = population_df.rename({"postcode_2021": "Postcode"}, axis = 1)
population_df = population_df.set_index("Postcode")
population_df.head()

,tot_avg_hh_size_21
Postcode,
3000,1.86
3002,1.87
3003,1.95
3004,1.84
3006,1.92


In [22]:
# Joining census population dataframe and facilities and amenities dataframe
df = population_df.join(postcode_property_df, on = "Postcode")
df.head()

,tot_avg_hh_size_21,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,,,,,,
3000,1.86,130.228671,1.482517,1.000000,1.377622,613.013986,1.601399,1.300699,0.356643
3002,1.87,302.665909,1.818182,1.000000,1.909091,628.068182,1.909091,1.318182,1.000000
3003,1.95,228.045312,1.718750,1.000000,2.000000,591.265625,1.796875,1.343750,0.609375
3004,1.84,429.717391,2.565217,1.130435,1.673913,564.804348,1.771739,1.423913,1.097826
3006,1.92,311.195918,2.663265,1.066327,2.306122,630.561224,1.785714,1.469388,0.647959


### Calculating liveability metric

In [23]:
# If the number of facilities is 0 (i.e. for number of bedrooms or number of parking spaces) replace 0 value with arbitrary small value epsilon
EPSILON = 10**(-6)
 
df.loc[df["Average # Beds"] == 0, "Average # Beds"] = EPSILON
df.loc[df["Average # Parking"] == 0, "Average # Parking"] = EPSILON

In [24]:
TOTAL_PROPERTIES = len(property_df)
TOTAL_SCHOOLS = len(schools_df)

# Counting the number of properties per postcode
num_properties = property_df.groupby("Postcode").size()

In [25]:
# Calcuting the contribution of each postcode to the total number of rental properties as a proportion
df["Property Proportion"] = num_properties / TOTAL_PROPERTIES
df["Schools Proportion"] = df["Average # of Schools"] / TOTAL_SCHOOLS
df.head()

,tot_avg_hh_size_21,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Schools Proportion
Postcode,,,,,,,,,,,
3000,1.86,130.228671,1.482517,1.000000,1.377622,613.013986,1.601399,1.300699,0.356643,0.010825,0.428981
3002,1.87,302.665909,1.818182,1.000000,1.909091,628.068182,1.909091,1.318182,1.000000,0.003331,0.439516
3003,1.95,228.045312,1.718750,1.000000,2.000000,591.265625,1.796875,1.343750,0.609375,0.004845,0.413762
3004,1.84,429.717391,2.565217,1.130435,1.673913,564.804348,1.771739,1.423913,1.097826,0.006964,0.395244
3006,1.92,311.195918,2.663265,1.066327,2.306122,630.561224,1.785714,1.469388,0.647959,0.014837,0.441260


In [26]:
# Calculating the average number of facilities per person in a household (for each postcode)
df["Beds per Person"] = df["Average # Beds"] / df["tot_avg_hh_size_21"]
df["Baths per Person"] = df["Average # Baths"] / df["tot_avg_hh_size_21"]
df["Parking per Person"] = df["Average # Parking"] / df["tot_avg_hh_size_21"]
df.head()

,tot_avg_hh_size_21,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Schools Proportion,Beds per Person,Baths per Person,Parking per Person
Postcode,,,,,,,,,,,,,,
3000,1.86,130.228671,1.482517,1.000000,1.377622,613.013986,1.601399,1.300699,0.356643,0.010825,0.428981,0.860967,0.699301,0.191744
3002,1.87,302.665909,1.818182,1.000000,1.909091,628.068182,1.909091,1.318182,1.000000,0.003331,0.439516,1.020904,0.704910,0.534759
3003,1.95,228.045312,1.718750,1.000000,2.000000,591.265625,1.796875,1.343750,0.609375,0.004845,0.413762,0.921474,0.689103,0.312500
3004,1.84,429.717391,2.565217,1.130435,1.673913,564.804348,1.771739,1.423913,1.097826,0.006964,0.395244,0.962902,0.773866,0.596645
3006,1.92,311.195918,2.663265,1.066327,2.306122,630.561224,1.785714,1.469388,0.647959,0.014837,0.441260,0.930060,0.765306,0.337479


In [27]:
# Calculating non-stadardised liveability metric
df["Liveability"] = ((1 / df["Average Duration to CBD"]) + df["Average Duration to Station"] + df["Average Distance to Park"] + 
                    df["Average Duration to Post Office"] + (df["Property Proportion"] * 100) + df["Beds per Person"] + df["Baths per Person"] + 
                    df["Parking per Person"] + df["Schools Proportion"])
df.head()

,tot_avg_hh_size_21,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Schools Proportion,Beds per Person,Baths per Person,Parking per Person,Liveability
Postcode,,,,,,,,,,,,,,,
3000,1.86,130.228671,1.482517,1.000000,1.377622,613.013986,1.601399,1.300699,0.356643,0.010825,0.428981,0.860967,0.699301,0.191744,7.131324
3002,1.87,302.665909,1.818182,1.000000,1.909091,628.068182,1.909091,1.318182,1.000000,0.003331,0.439516,1.020904,0.704910,0.534759,7.763747
3003,1.95,228.045312,1.718750,1.000000,2.000000,591.265625,1.796875,1.343750,0.609375,0.004845,0.413762,0.921474,0.689103,0.312500,7.544455
3004,1.84,429.717391,2.565217,1.130435,1.673913,564.804348,1.771739,1.423913,1.097826,0.006964,0.395244,0.962902,0.773866,0.596645,8.796991
3006,1.92,311.195918,2.663265,1.066327,2.306122,630.561224,1.785714,1.469388,0.647959,0.014837,0.441260,0.930060,0.765306,0.337479,9.996757


In [28]:
# Calculating standardised liveability metric
min_liveability = df.sort_values(by = "Liveability").head(1)["Liveability"].tolist()[0]
max_liveability = df.sort_values(by = "Liveability", ascending = False).head(1)["Liveability"].tolist()[0]

df["Standardised Liveability"] = (df["Liveability"] - min_liveability) / (max_liveability - min_liveability)

# Printing the top 10 most liveable suburbs
df.sort_values(by = "Standardised Liveability", ascending = False).head(10)

,tot_avg_hh_size_21,Average Duration to CBD,Average Duration to Station,Average Distance to Park,Average Duration to Post Office,Average # of Schools,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Schools Proportion,Beds per Person,Baths per Person,Parking per Person,Liveability,Standardised Liveability
Postcode,,,,,,,,,,,,,,,,
3213,2.49,3339.2,6.0,4.75,6.0,76.75,3.500000,1.750000,9.250000,0.000303,0.053709,1.405622,0.702811,3.714859,22.657582,1.000000
3688,2.40,12071.3,6.0,6.00,6.0,22.00,5.000000,3.000000,3.000000,0.000076,0.015395,2.083333,1.250000,1.250000,22.606382,0.997138
3808,2.82,3331.1,6.0,6.00,6.0,110.00,5.000000,4.000000,2.000000,0.000076,0.076977,1.773050,1.418440,0.709220,21.985556,0.962440
3670,2.40,7969.4,6.0,6.00,6.0,11.00,4.000000,1.000000,4.000000,0.000076,0.007698,1.666667,0.416667,1.666667,21.765393,0.950135
3799,2.30,5608.3,6.0,6.00,6.0,12.00,3.000000,3.000000,2.000000,0.000076,0.008397,1.304348,1.304348,0.869565,21.494407,0.934989
3249,2.46,7504.5,6.0,6.00,6.0,9.00,3.000000,1.000000,4.000000,0.000076,0.006298,1.219512,0.406504,1.626016,21.266034,0.922225
3659,2.47,4946.6,6.0,6.00,6.0,4.00,4.000000,2.000000,2.000000,0.000076,0.002799,1.619433,0.809717,0.809717,21.249438,0.921298
3461,2.35,4930.7,6.0,6.00,6.0,18.00,2.000000,1.500000,3.000000,0.000151,0.012596,0.851064,0.638298,1.276596,20.793897,0.895837
3498,2.46,22067.9,6.0,6.00,6.0,26.00,3.166667,1.666667,1.833333,0.000454,0.018195,1.287263,0.677507,0.745257,20.773687,0.894708


In [29]:
df = df.reset_index()
liveability_df = df[["Postcode", "Standardised Liveability"]]

### Graphing standardised liveability

In [30]:
# Creating geoJSON file of postcode and geometry coordinates 
sf = gpd.read_file("../data/raw/external/POA_2021_AUST_GDA2020/POA_2021_AUST_GDA2020.shp")
postcodes = pd.read_csv("../data/raw/external/postcode.csv", names = ["POA_CODE21", "Name", "Area"])
postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)

# Converting the geometry shaape to to latitude and longitude
# TAKEN FROM TUTE 2 NOTEBOOK
sf["geometry"] = sf["geometry"].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf = sf[sf["POA_CODE21"].between("3000", "4000")]
sf["POA_CODE21"] = sf["POA_CODE21"].astype(int)



gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

In [31]:
# (y, x) since we want (lat, long)
gdf["centroid"] = gdf["geometry"].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[["Name", "POA_CODE21", "centroid"]].head()

liveability_df.rename({"Postcode": "POA_CODE21"})

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

c = folium.Choropleth(
            geo_data = geoJSON, # geoJSON 
            name = "choropleth", # name of plot
            data = liveability_df, # data source
            columns = ["Postcode", "Standardised Liveability"], # the columns required
            key_on = "properties.POA_CODE21", # this is from the geoJSON's properties
            fill_color = "YlOrRd", # color scheme
            nan_fill_color = "grey",
            legend_name = "Liveability"
        )

c.add_to(m)
m.save(f"../plots/liveability_heatmap")

In [35]:
# Creating GeoPandas visualisation of top 10 most liveable postcodes
liveability_df.to_csv("../data/curated/liveability.csv", index=False)

postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)
postcodes = postcodes.loc[postcodes["POA_CODE21"].isin(liveability_df["Postcode"].to_list())]

liveability_df.sort_values(by="Standardised Liveability", ascending = False).head(10)


,Postcode,Standardised Liveability
184,3213,1.000000
485,3688,0.997138
571,3808,0.962440
475,3670,0.950135
564,3799,0.934989
214,3249,0.922225
467,3659,0.921298
349,3461,0.895837
372,3498,0.894708
240,3285,0.886109


In [33]:
# Create a heatmap displaying the top most liveable postcodes
gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

m.add_child(folium.Choropleth(geo_data = geoJSON, name = "choropleth",))
m.save(f"../plots/most_liveable_heatmap")